# Importing Dependencies

In [82]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, lit, count, desc
import pandas as pd
import time

# Combine credits and movies

In [83]:
start_time = time.time()
spark = SparkSession.builder \
    .appName("CSV Merge") \
    .getOrCreate()

movies_df = spark.read.csv('movies_metadata.csv', header=True, inferSchema=True) 
credits_df = spark.read.csv('credits.csv', header=True, inferSchema=True)

merged_df = movies_df.join(credits_df, on=['id'], how='inner')
end_time = time.time()
# Calculate the execution time
execution_time = end_time - start_time

# Print the execution time
print("Execution Time: {} seconds".format(execution_time))

Execution Time: 1.1713840961456299 seconds


# Reading the Dataset

In [84]:
movies = merged_df.select([col(column).alias(column) for column in merged_df.columns])

# Checking for missing values (per column)

In [85]:
total_rows = movies.count()
missing_values_counts = [sum(col(column).isNull().cast("int")).alias(column) for column in movies.columns]
missing_values_df = movies.select(missing_values_counts)
missing_values_percentages = missing_values_df.select(
    *[(col(column) / total_rows * 100).alias(column) for column in missing_values_df.columns]
)
missing_values_df = missing_values_df.union(missing_values_percentages)
missing_values_df = missing_values_df.toPandas().T
missing_values_df = missing_values_df.rename(columns={0: "Missing Values", 1: "Missing Values Percentage"})
missing_values_df

,Missing Values,Missing Values Percentage
id,0.0,0.000000
adult,0.0,0.000000
belongs_to_collection,27959.0,91.837472
budget,0.0,0.000000
genres,0.0,0.000000
homepage,25212.0,82.814348
imdb_id,15.0,0.049271
original_language,11.0,0.036132
original_title,0.0,0.000000
overview,769.0,2.525949


# Dropping Columns

In [86]:
columns_to_drop = [
    'belongs_to_collection', 'homepage', 'tagline', 'imdb_id', 'id', 'poster_path'
]


movies = movies.drop(*columns_to_drop)

# Checking for unique values (by column)

In [87]:
unique_values = {}
for column in movies.columns:
    unique_values[column] = movies.select(column).distinct()

# Show unique values for each column
for column, values in unique_values.items():
    print(f"Unique values for column '{column}':")
    values.show()
    print(f"Number of unique values for column '{column}':", values.count())
    print('\n')

Unique values for column 'adult':
+-----+
|adult|
+-----+
|False|
| True|
+-----+

Number of unique values for column 'adult': 2


Unique values for column 'budget':
+--------+
|  budget|
+--------+
|36000000|
|  150000|
|34000000|
| 4940939|
|45000000|
| 1250000|
|  750000|
|37000000|
|48000000|
|14000000|
|15000000|
|      30|
|  935000|
|   90000|
|     250|
|   67000|
|       8|
|16400000|
|  609000|
| 7500000|
+--------+
only showing top 20 rows

Number of unique values for column 'budget': 820


Unique values for column 'genres':
+--------------------+
|              genres|
+--------------------+
|[{'id': 12, 'name...|
|[{'id': 18, 'name...|
|[{'id': 53, 'name...|
|[{'id': 14, 'name...|
|[{'id': 28, 'name...|
|[{'id': 28, 'name...|
|[{'id': 99, 'name...|
|[{'id': 53, 'name...|
|[{'id': 12, 'name...|
|[{'id': 35, 'name...|
|[{'id': 35, 'name...|
|[{'id': 14, 'name...|
|[{'id': 18, 'name...|
|[{'id': 12, 'name...|
|[{'id': 12, 'name...|
|[{'id': 99, 'name...|
|[{'id': 10749, 'n...

# Most common value for each column

In [88]:
most_common_values = {}
for column in movies.columns:
    column_counts = movies.groupBy(column).agg(count("*").alias("count"))
    total_count = movies.count()
    column_counts_with_percentage = column_counts.withColumn("percentage", (col("count") / total_count) * 100)
    column_counts_with_percentage = column_counts_with_percentage.orderBy(desc("count"))
    most_common_values[column] = (column_counts_with_percentage.first()[column],
                                  column_counts_with_percentage.first()["percentage"])

for column, (value, percentage) in most_common_values.items():
    print(f"For column '{column}': {value}")
    print(f"Percentage: {percentage:.2f}%")

For column 'adult': False
Percentage: 99.98%
For column 'budget': 0
Percentage: 85.07%
For column 'genres': [{'id': 18, 'name': 'Drama'}]
Percentage: 11.19%
For column 'original_language': en
Percentage: 69.03%
For column 'original_title': Blackout
Percentage: 0.04%
For column 'overview': None
Percentage: 2.53%
For column 'popularity': 0.0
Percentage: 0.21%
For column 'production_companies': []
Percentage: 29.61%
For column 'production_countries': [{'iso_3166_1': 'US', 'name': 'United States of America'}]
Percentage: 31.64%
For column 'release_date': [{'iso_3166_1': 'US', 'name': 'United States of America'}]
Percentage: 0.93%
For column 'revenue': 0
Percentage: 81.55%
For column 'runtime': 90.0
Percentage: 5.63%
For column 'spoken_languages': [{'iso_639_1': 'en', 'name': 'English'}]
Percentage: 42.37%
For column 'status': Released
Percentage: 90.89%
For column 'title': None
Percentage: 1.65%
For column 'video': False
Percentage: 91.70%
For column 'vote_average': 0.0
Percentage: 7.37%
F

# Filtering and dropping further

In [89]:
filtered_movies = movies.filter(movies.adult == 'False')
filtered_movies = filtered_movies.filter(movies.status == 'Released')
filtered_movies = filtered_movies.filter(movies.video == False)

columns_to_drop = [
    'adult', 'status', 'video'
]

filtered_movies = filtered_movies.drop(*columns_to_drop)

# Looking for duplicates and removing them

In [90]:
duplicate_rows = filtered_movies.groupBy(filtered_movies.columns).count().where(col("count") > 1)
print('Number of duplicate rows: ', duplicate_rows.count())
filtered_movies = filtered_movies.dropDuplicates()

Number of duplicate rows:  32


# A Summary of the dataset

In [92]:
filtered_movies.describe().show()

+-------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-------------------------+--------------------+------------+-------------------+-----------------+--------------------+--------------------+-----------------+------------------+--------------------+--------------------+
|summary|              budget|              genres|original_language|      original_title|            overview|          popularity|     production_companies|production_countries|release_date|            revenue|          runtime|    spoken_languages|               title|     vote_average|        vote_count|                cast|                crew|
+-------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-------------------------+--------------------+------------+-------------------+-----------------+--------------------+--------------------+-----------------+------